# Adapted jupyter-notebook for running suite2p algorithm on 2P data

In [1]:
from pathlib import Path
from tempfile import TemporaryDirectory
import matplotlib.pyplot as plt
import numpy as np
import os
import tifffile
import xmltodict
import suite2p
import fastplotlib as fpl
import json

# Figure Style settings for notebook.
import matplotlib as mpl
mpl.rcParams.update({
    'axes.spines.left': False,
    'axes.spines.bottom': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'legend.frameon': False,
    'figure.subplot.wspace': .01,
    'figure.subplot.hspace': .01,
    'figure.figsize': (9, 9),
    'ytick.major.left': False,
})
jet = mpl.cm.get_cmap('jet')
jet.set_bad(color='k')

/var/folders/f3/11w5x6bs32gd1xc4lh41qdnr0000gp/T/ipykernel_64644/413991043.py:25: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  jet = mpl.cm.get_cmap('jet')


# Set `movie_path` and let the rest work

When you are finished, open Suite2p_quickViz, change the file path and visualize the movie overlaid with the components. You should toggle between the suite2p GUI and the fastplotlib plots

In [7]:
# CHANGE ME
movie_path = r'/Users/js0403/miniscope/PFC-Neurons/122D/AAV2/3-Syn-GCaMP8f/2024_03_21/15_01_43/miniscopeDeviceName/full_movie.tif'
movie_orig = tifffile.memmap(movie_path)
ops = np.load(os.path.join(os.path.split(movie_path)[0],'1P_ops.npy'),allow_pickle=True).item()
ops = suite2p.default_ops()

# 1P
#ops['fs']=10
'''
ops['tau']=0.7
ops['two_step_registration']=True # for low SNR
ops['smooth_sigma_time']=2 # 1-2 for low SNR
ops['block_size']=[128,128]
ops['snr_thresh']=1.5
ops['threshold_scaling']=1.0 # how much the roi has to stand out from noise
ops['high_pass']=5 # running mean subtraction across time with a window size of 'high_pass'. Less than 10 for 1P (try higher!)
ops['1Preg']=True
ops['spatial_hp_reg']=42
ops['pre_smooth']=2 # gaussian applied before spatial high pass filtering
ops['spatial_taper']=40
'''

"\nops['tau']=0.7\nops['two_step_registration']=True # for low SNR\nops['smooth_sigma_time']=2 # 1-2 for low SNR\nops['block_size']=[128,128]\nops['snr_thresh']=1.5\nops['threshold_scaling']=1.0 # how much the roi has to stand out from noise\nops['high_pass']=5 # running mean subtraction across time with a window size of 'high_pass'. Less than 10 for 1P (try higher!)\nops['1Preg']=True\nops['spatial_hp_reg']=42\nops['pre_smooth']=2 # gaussian applied before spatial high pass filtering\nops['spatial_taper']=40\n"

In [8]:
#___________________________________________#

# automatically retrieve the sampling rate
root_path = os.path.split(movie_path)[0]
movie_name = os.path.split(movie_path)[1]
root_contents = os.listdir(root_path)
metadata_dir = os.path.join(root_path,[i for i in root_contents if '.json' in i][0])
metadata = json.load(open(metadata_dir))
fr = metadata['frameRate']

# default ops
#ops = suite2p.default_ops()
ops['fs']=fr
ops['save_NWB']=False
print(ops)

# set db, this overrides the ops variable
db = {
    'data_path': [root_path],
    'save_path0': root_path,
    'tiff_list': [movie_name],
}
db

# run suite2p algorithm
output_ops = suite2p.run_s2p(ops=ops, db=db)

{'suite2p_version': '0.14.2', 'look_one_level_down': 0.0, 'fast_disk': [], 'delete_bin': False, 'mesoscan': False, 'bruker': False, 'bruker_bidirectional': False, 'h5py': [], 'h5py_key': 'data', 'nwb_file': '', 'nwb_driver': '', 'nwb_series': '', 'save_path0': '', 'save_folder': [], 'subfolders': [], 'move_bin': False, 'nplanes': 1, 'nchannels': 1, 'functional_chan': 1, 'tau': 1.0, 'fs': 10, 'force_sktiff': False, 'frames_include': -1, 'multiplane_parallel': 0.0, 'ignore_flyback': [], 'preclassify': 0.0, 'save_mat': False, 'save_NWB': False, 'combined': 1.0, 'aspect': 1.0, 'do_bidiphase': False, 'bidiphase': 0.0, 'bidi_corrected': False, 'do_registration': 1, 'two_step_registration': 0.0, 'keep_movie_raw': False, 'nimg_init': 300, 'batch_size': 500, 'maxregshift': 0.1, 'align_by_chan': 1, 'reg_tif': False, 'reg_tif_chan2': False, 'subpixel': 10, 'smooth_sigma_time': 0.0, 'smooth_sigma': 4.0, 'th_badframes': 1.0, 'norm_frames': True, 'force_refImg': False, 'pad_fft': False, 'nonrigid': 

/Users/js0403/anaconda3/envs/suite2p/lib/python3.9/site-packages/suite2p/detection/sparsedetect.py:286: UserWarning: Spatial scale estimation failed.  Setting spatial scale to 1 in order to continue.
  warn(


NOTE: FORCED spatial scale ~6 pixels, time epochs 1.00, threshold 5.00 
0 ROIs, score=30.56
Detected 644 ROIs, 6.88 sec
After removing overlaps, 627 ROIs remain
----------- Total 8.08 sec.
----------- EXTRACTION
Masks created, 0.68 sec.
Extracted fluorescence from 627 ROIs in 1997 frames, 1.27 sec.
----------- Total 1.99 sec.
----------- CLASSIFICATION
['skew', 'npix_norm', 'compact']
----------- SPIKE DECONVOLUTION
----------- Total 0.05 sec.
Plane 0 processed in 10.49 sec (can open in GUI).
total = 10.55 sec.
TOTAL RUNTIME 10.55 sec
